# Creating WCM File Component using Python
First specify the libraries needed.  In this example the Requests library was used to communicate with the Portal/WCM server.  
Note: This example uses [Python f-strings](https://realpython.com/python-f-strings/) which were introduced in v3.6.

In [1]:
import json
import requests

You'll need the URL for the WCM REST API.  I split this into two variables, and you'll see why a little later on.  Replace the `hostname:port` with your values in the `baseURL` variable.

In [2]:
baseURL = 'http://portal9.home.parrill.net:10039'
restURL = f'{baseURL}/wps/mycontenthandler/wcmrest'

The first step is to get the UUID of the library which will contain the file component.  While this can be done using the user interface, it's more fun to do it programmatically.  In this example we'll search by the library title.  
A library named `Test Upload` had been previously created.

In [3]:
# Initialize our Requests session parameters.  You'll want to replace the credentials with ones for your own environment
s = requests.Session()
s.auth=('wpsadmin', 'password')  # Normally you wouldn't hard-code something like this!

# Use the Query function to find the title we're looking for.  If you know the exact name you can use
# the 'title' query parameter, or you can use 'titlelike' if you have a partial name.
# If mulitple matches are found this example is only using the first match
libName = 'Test Upload'
r = s.get(f'{restURL}/query?type=Library&titlelike={libName}&mime-type=application/json')
rDict = json.loads(r.text)
libID = rDict['feed']['entry'][0]['id'].split(':')[1]

print(r.text)
print(f'The library ID is: {libID}')

{
    "feed" :
    {
        "id" : "wcmrest:query?mime-type=application%2Fjson&titlelike=Test%20Upload&type=Library",
        "title" : "wcmrest:query?mime-type=application%2Fjson&titlelike=Test%20Upload&type=Library",
        "updated" : "Fri, 09 Nov 2018 22:51:50.301Z",
        "entry" :
        [
            {
                "id" : "wcmrest:38606f98-ffb1-4b8e-bd81-3a1692b748fe",
                "title" : "Test Upload",
                "summary" : null,
                "type" : "Library",
                "updated" : "Fri, 09 Nov 2018 18:47:04.903Z",
                "link" :
                [
                    {
                        "rel" : "edit",
                        "href" : "/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Library/38606f98-ffb1-4b8e-bd81-3a1692b748fe",
                        "lang" : "en",
                        "label" : "Edit"
                    },
                    {
                        "rel" : "alternate",
                  

In addition to the library ID, the library link will also be required.  This could be parsed out of the info above, but in this example we're making a call to the library object to get additional information about it.  
The `<link rel="self" ... >` tag is the one to be parsed for the URL.

In [4]:
r = s.get(f'{restURL}/Library/{libID}?mime-type=application/json')
rDict = json.loads(r.text)
libLnk = next(item for item in rDict['entry']['link'] if item["rel"] == "self")['href']

print(r.text)
print(f'The library link is: {libLnk}')

{
    "entry" :
    {
        "id" : "wcmrest:38606f98-ffb1-4b8e-bd81-3a1692b748fe",
        "title" : "Test Upload",
        "summary" : null,
        "name" : "test upload",
        "type" : "Library",
        "updated" : "Fri, 09 Nov 2018 18:47:04.903Z",
        "link" :
        [
            {
                "rel" : "self",
                "href" : "/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Library/38606f98-ffb1-4b8e-bd81-3a1692b748fe",
                "lang" : "en",
                "label" : "Read"
            },
            {
                "rel" : "edit",
                "href" : "/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Library/38606f98-ffb1-4b8e-bd81-3a1692b748fe",
                "lang" : "en",
                "label" : "Edit"
            },
            {
                "rel" : "delete",
                "href" : "/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Library/38606f98-ffb1-4b8e-bd81-3a1692b748fe",


Notice the library link is a relative path starting with `/wps/mycontenthandler/...`

Next, create the XML which will be sent with the request to create the file component.  This example just creates a raw string with the desired inputs.  Be sure to specify the `href=` parameter using the link to your library.  Adjust the `title`, `name`, and `description` fields as needed.

In [5]:
fileXML = f'<entry xmlns="http://www.w3.org/2005/Atom" xmlns:wcm="wcm/namespace"> \
        <title>Programmatic Test File Component 2</title> \
        <link rel="library" href="{libLnk}"/> \
        <wcm:name>ProgTestFileComponent2</wcm:name> \
        <wcm:description>This is test file component uploaded from Python</wcm:description> \
    </entry>'

Now call the POST interface to create the new file component.  Note that the file is not yet included.  Be sure to save the UUID of the new file component since that will be needed shortly.

In [6]:
r = s.post(f'{restURL}/LibraryFileComponent?mime-type=application/json',
                data = fileXML,
                headers={'Content-Type':'application/atom+xml'})
rDict = json.loads(r.text)
fileID = rDict['entry']['id'].split(':')[1]

print(r.text)
print(f'The file ID is: {fileID}')

{
    "entry" :
    {
        "id" : "wcmrest:33e0a323-4ebb-46d4-8d11-cc23dd6e4ec1",
        "title" :
        {
            "lang" : "en",
            "value" : "Programmatic Test File Component 2"
        },
        "summary" :
        {
            "lang" : "en"
        },
        "name" : "ProgTestFileComponent2",
        "type" : "LibraryFileComponent",
        "updated" : "Fri, 09 Nov 2018 22:51:50.434Z",
        "created" : "Fri, 09 Nov 2018 22:51:50.434Z",
        "author" :
        [
            {
                "distinguishedName" : "uid=wpsadmin,o=defaultWIMFileBasedRealm",
                "uri" : "/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/um/users/profiles/Z9eAeG1CA6R8643D4JM07GPD8MMG663PAJM4C43D4JM4CM9P43O86OPO83O4643",
                "name" : "wpsadmin"
            }
        ],
        "owner" :
        [
            {
                "distinguishedName" : "uid=wpsadmin,o=defaultWIMFileBasedRealm",
                "uri" : "/wps/mycontenthandler/!ut/p/dig

If successful the file may now be added to the component using the PUT command.  The file is added as binary data, and be sure to specify the file's UUID in the command.
This example uses a local file named `TEST.PDF`.

In [7]:
with open('TEST.PDF', mode='rb') as fh:
    mydata = fh.read()
r = s.put(f'{restURL}/LibraryFileComponent/{fileID}?mime-type=application/json',
          data = mydata,
          headers={'Content-Type':'application/pdf'})

print(r.text)

{
    "entry" :
    {
        "id" : "wcmrest:33e0a323-4ebb-46d4-8d11-cc23dd6e4ec1",
        "title" :
        {
            "lang" : "en",
            "value" : "Programmatic Test File Component 2"
        },
        "summary" :
        {
            "lang" : "en"
        },
        "name" : "ProgTestFileComponent2",
        "type" : "LibraryFileComponent",
        "updated" : "Fri, 09 Nov 2018 22:51:50.984Z",
        "created" : "Fri, 09 Nov 2018 22:51:50.434Z",
        "author" :
        [
            {
                "distinguishedName" : "uid=wpsadmin,o=defaultWIMFileBasedRealm",
                "uri" : "/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/um/users/profiles/Z9eAeG1CA6R8643D4JM07GPD8MMG663PAJM4C43D4JM4CM9P43O86OPO83O4643",
                "name" : "wpsadmin"
            }
        ],
        "owner" :
        [
            {
                "distinguishedName" : "uid=wpsadmin,o=defaultWIMFileBasedRealm",
                "uri" : "/wps/mycontenthandler/!ut/p/dig

Verify the item was created and the file is attached by going into the WCM authoring portlet and looking at the Components folder inside the library:  
![alt text](Library_Item_Created.jpg)  
View the details for the new file component:  
![alt text](Library_Item_Details.jpg) 

## Retrieve the item
The new file component can also be easily retrieved using the REST API as well.  
The [WCM Query service](https://www.ibm.com/support/knowledgecenter/SSHRKX_8.5.0/mp/wcm/wcm_rest_adhoc.html) 
can be used to find the UUID of the item using a variety of search parameters.  In this case the UUID is already known from the file component created earlier.  
A WCM component can be read by sending a GET request to the following URI: `/<library-component-api-type>/<itemuuid>`  
The file component uses a type of `LibraryFileComponent`.

In [8]:
r = s.get(f'{restURL}/LibraryFileComponent/{fileID}?mime-type=application/json')

print(r.text)

{
    "entry" :
    {
        "id" : "wcmrest:33e0a323-4ebb-46d4-8d11-cc23dd6e4ec1",
        "title" :
        {
            "lang" : "en",
            "value" : "Programmatic Test File Component 2"
        },
        "summary" :
        {
            "lang" : "en"
        },
        "name" : "ProgTestFileComponent2",
        "type" : "LibraryFileComponent",
        "updated" : "Fri, 09 Nov 2018 22:51:50.984Z",
        "created" : "Fri, 09 Nov 2018 22:51:50.434Z",
        "author" :
        [
            {
                "distinguishedName" : "uid=wpsadmin,o=defaultWIMFileBasedRealm",
                "uri" : "/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/um/users/profiles/Z9eAeG1CA6R8643D4JM07GPD8MMG663PAJM4C43D4JM4CM9P43O86OPO83O4643",
                "name" : "wpsadmin"
            }
        ],
        "owner" :
        [
            {
                "distinguishedName" : "uid=wpsadmin,o=defaultWIMFileBasedRealm",
                "uri" : "/wps/mycontenthandler/!ut/p/dig

Parse the data to get the URL for the actual file attachment.

In [9]:
rDict = json.loads(r.text)
lnk = next(item for item in rDict['entry']['link'] if item["rel"] == "self")['href']

print(f'The link to the file is: {lnk}')

The link to the file is: /wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/LibraryFileComponent/33e0a323-4ebb-46d4-8d11-cc23dd6e4ec1


In [10]:
print(f'The full url is: {baseURL}{lnk}')

The full url is: http://portal9.home.parrill.net:10039/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/LibraryFileComponent/33e0a323-4ebb-46d4-8d11-cc23dd6e4ec1
